In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
data = pd.read_csv('movie_bd_v5.csv')

data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1274,tt1924396,13500000,19255873,La migliore offerta,Geoffrey Rush|Jim Sturgess|Sylvia Hoeks|Donald...,Giuseppe Tornatore,A Master of Possession. A Crime of Obsession,Virgil Oldman is a world renowned antiques exp...,124,Drama|Romance|Crime|Mystery,Paco Cinematografica|Warner Bros.|Regione Lazi...,1/1/2013,7.7,2013
255,tt0878804,29000000,309208309,The Blind Side,Sandra Bullock|Quinton Aaron|Kathy Bates|Tim M...,John Lee Hancock,Based on the extraordinary true story,"Oversized African-American, Michael Oher, the ...",129,Drama,Alcon Entertainment|Left Tackle Pictures|Zucke...,11/20/2009,7.1,2009
1509,tt0469641,63000000,162945894,World Trade Center,Nicolas Cage|Maria Bello|Maggie Gyllenhaal|Mic...,Oliver Stone,The World Saw Evil That Day. Two Men Saw Somet...,"On September, 11th 2001, after the terrorist a...",128,Drama|History|Thriller,Paramount Pictures|Double Feature Films|Interm...,8/9/2006,5.9,2006
1319,tt0371724,50000000,104478416,The Hitchhiker's Guide to the Galaxy,Martin Freeman|Zooey Deschanel|Sam Rockwell|Mo...,Garth Jennings,Don't leave Earth without it.,Mere seconds before the Earth is to be demolis...,109,Adventure|Comedy|Family|Science Fiction,Spyglass Entertainment|Everyman Pictures|Hamme...,1/15/2005,6.6,2005
188,tt2357291,103000000,500188435,Rio 2,Jesse Eisenberg|Anne Hathaway|Leslie Mann|Jami...,Carlos Saldanha,"He's villainous, she's venomous.","It's a jungle out there for Blu, Jewel and the...",102,Animation|Adventure|Comedy|Family,Blue Sky Studios|Twentieth Century Fox Animation,3/19/2014,6.5,2014


In [3]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


In [4]:
# Предобработка
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


In [5]:
answers = {} # создадим словарь для ответов


# тут другие ваши предобработки колонок например:
def split_str(cast : str) -> str:
    '''Возвращает массив строк, разбитых по символу'''
    return [str(actor) for actor in cast.split('|')]

def is_profit(row : pd.DataFrame) -> bool:
    '''Индикатор успешности фильма'''
    if row.budget < row.revenue:
        return True
    else:
        return False

def count_genres(data : pd.DataFrame) -> list:
    '''Подсчет жанров фильма'''
    c = Counter()
    for row in data.list_genre:
        for genre in row:
            c[genre] += 1
    return c

# Первый вариант разбитя по символу
data['list_cast'] = data.cast.apply(split_str)
#data.production_companies = data.production_companies.apply(split_str)

#data.genres = data.genres.apply(split_str)

# посчитать пропущенные значения
data.isna().sum()

#Привести к типу Datetime
data.release_date = pd.to_datetime(data.release_date)
# Добавить колонку окупаемости
data['is_profit'] = data.apply(is_profit, axis = 1)
# Добавляем колонку прибильности/убытка фильма
data['profit'] = data.apply(lambda x: x['revenue'] - x['budget'], axis = 1)
# Второй вариант разбитя по символу
data['list_director'] = data.director.str.split('|')
data['list_genre'] = data.genres.str.split('|')
data['list_company'] = data.production_companies.str.split('|')

# Выносим месяцы создания фильма в отдельный столбец
data['month'] = data.release_date.map(lambda x: x.month)

data.info()
#the time given in the dataset is in string format.
#So we need to change this in datetime format
# ...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   imdb_id               1889 non-null   object        
 1   budget                1889 non-null   int64         
 2   revenue               1889 non-null   int64         
 3   original_title        1889 non-null   object        
 4   cast                  1889 non-null   object        
 5   director              1889 non-null   object        
 6   tagline               1889 non-null   object        
 7   overview              1889 non-null   object        
 8   runtime               1889 non-null   int64         
 9   genres                1889 non-null   object        
 10  production_companies  1889 non-null   object        
 11  release_date          1889 non-null   datetime64[ns]
 12  vote_average          1889 non-null   float64       
 13  release_year      

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [6]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
#answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа +
answers['1'] = '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)' 
# если ответили верно, можете добавить комментарий со значком "+"

In [7]:
# тут пишем ваш код для решения данного вопроса:
data[data.budget == data.budget.max()][['imdb_id','budget','original_title']]

,imdb_id,budget,original_title
723,tt1298650,380000000,Pirates of the Caribbean: On Stranger Tides


ВАРИАНТ 2

In [8]:
# можно добавлять разные варианты решения
data.sort_values(by=['budget'], ascending=False).head(1)[['imdb_id','budget','original_title','runtime']]

,imdb_id,budget,original_title,runtime
723,tt1298650,380000000,Pirates of the Caribbean: On Stranger Tides,136


# 2. Какой из фильмов самый длительный (в минутах)?

In [9]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
answers['2'] = '2. Gods and Generals (tt0279111)'

In [10]:
data[data.runtime == data.runtime.max()][['imdb_id','budget','original_title','runtime']]

,imdb_id,budget,original_title,runtime
1157,tt0279111,56000000,Gods and Generals,214


In [11]:
# вариант второй
data.sort_values(by=['runtime'], ascending=False).head(1)[['imdb_id','budget','original_title','runtime']]


,imdb_id,budget,original_title,runtime
1157,tt0279111,56000000,Gods and Generals,214


# 3. Какой из фильмов самый короткий (в минутах)?





In [12]:
answers['3'] = '3. Winnie the Pooh  (tt1449283)'

In [13]:
data[data.runtime == data.runtime.min()][['imdb_id','budget','original_title','runtime']]

,imdb_id,budget,original_title,runtime
768,tt1449283,30000000,Winnie the Pooh,63


In [14]:
# Второй вариант 
data.sort_values(by=['runtime']).head(1)[['imdb_id','budget','original_title','runtime']]

,imdb_id,budget,original_title,runtime
768,tt1449283,30000000,Winnie the Pooh,63


In [15]:
# 4. Какова средняя длительность фильмов?
answers['4'] = '2. 110'

In [16]:
round(data.runtime.mean())

110.0

# 5. Каково медианное значение длительности фильмов? 

In [17]:
answers['5'] = '1. 107'

In [18]:
round(data.runtime.median())

107.0

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [19]:
answers['6'] = '5. Avatar (tt0499549)'

In [20]:
# лучше код получения столбца profit вынести в Предобработку что в начале
data[(data.is_profit == True) & (data.profit == max(data.profit))][['imdb_id','budget','revenue','profit','original_title','runtime']]

,imdb_id,budget,revenue,profit,original_title,runtime
239,tt0499549,237000000,2781505847,2544505847,Avatar,162


# 7. Какой фильм самый убыточный? 

In [21]:
answers['7'] = '5. The Lone Ranger (tt1210819)'

In [22]:
data[(data.is_profit == False) & (data.profit == min(data.profit))][['imdb_id','budget','revenue','profit','original_title','runtime']]

,imdb_id,budget,revenue,profit,original_title,runtime
1245,tt1210819,255000000,89289910,-165710090,The Lone Ranger,149


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [23]:
answers['8'] = '1. 1478'

In [24]:
data[data.is_profit == True].is_profit.count()

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [25]:
answers['9'] = '4. The Dark Knight (tt0468569)'

In [26]:
max_revenue = data.groupby(['release_year'])['revenue'].max()
data[data.revenue == max_revenue.loc[2008]]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,release_date,vote_average,release_year,list_cast,is_profit,profit,list_director,list_genre,list_company,month
599,tt0468569,185000000,1001921825,The Dark Knight,Christian Bale|Michael Caine|Heath Ledger|Aaro...,Christopher Nolan,Why So Serious?,Batman raises the stakes in his war on crime. ...,152,Drama|Action|Crime|Thriller,...,2008-07-16,8.1,2008,"[Christian Bale, Michael Caine, Heath Ledger, ...",True,816921825,[Christopher Nolan],"[Drama, Action, Crime, Thriller]","[DC Comics, Legendary Pictures, Warner Bros., ...",7


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [27]:
answers['10'] = '5. The Lone Ranger (tt1210819)'

In [28]:
mins_profit = data.groupby(['release_year'])[['profit']].min()
mins_our_years = mins_profit.loc[[2012,2013,2014]].min()
data[data.profit == mins_our_years[0]]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,release_date,vote_average,release_year,list_cast,is_profit,profit,list_director,list_genre,list_company,month
1245,tt1210819,255000000,89289910,The Lone Ranger,Johnny Depp|Armie Hammer|William Fichtner|Hele...,Gore Verbinski,Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,Action|Adventure|Western,...,2013-07-03,6.0,2013,"[Johnny Depp, Armie Hammer, William Fichtner, ...",False,-165710090,[Gore Verbinski],"[Action, Adventure, Western]","[Walt Disney Pictures, Jerry Bruckheimer Films...",7


# 11. Какого жанра фильмов больше всего?

In [29]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале
# либо функцией counter посчитать
unique_genres = count_genres(data)
print(unique_genres)
answers['11'] = '3. Drama'

Counter({'Drama': 782, 'Comedy': 683, 'Thriller': 596, 'Action': 582, 'Adventure': 415, 'Crime': 315, 'Romance': 308, 'Family': 260, 'Science Fiction': 248, 'Fantasy': 222, 'Horror': 176, 'Mystery': 168, 'Animation': 139, 'Music': 64, 'History': 62, 'War': 58, 'Western': 19, 'Documentary': 8, 'Foreign': 2})


ВАРИАНТ 2

In [30]:
genres_exploded = data.explode('list_genre')
genres_exploded.list_genre.value_counts(sort=True).head(1)

Drama    782
Name: list_genre, dtype: int64

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [31]:
answers['12'] = '1. Drama'
profit_films = data[data.profit > 0]
profit_films_exploded = profit_films.explode('list_genre')
profit_films_exploded.list_genre.value_counts().head(1)

Drama    560
Name: list_genre, dtype: int64

# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [32]:
answers['13'] = '5. Peter Jackson'
data.groupby(['director'])['revenue'].sum().sort_values(ascending=False).head(1)

director
Peter Jackson    6490593685
Name: revenue, dtype: int64


# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [33]:
answers['14'] = '3. Robert Rodriguez'
data_action = data[data.genres.str.contains('Action')][['list_director']]
data_action_exploded = data_action.explode('list_director')
data_action_exploded.list_director.value_counts().head(1)

Robert Rodriguez    9
Name: list_director, dtype: int64

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [34]:
answers['15'] = '3. Chris Hemsworth'
data_2012 = data[(data.release_year == 2012) & (data.profit > 0)]
data_exploded = data_2012.explode('list_cast')
data_exploded.groupby(by = 'list_cast')['revenue'].agg('sum').sort_values(ascending=False).head(1)

list_cast
Chris Hemsworth    1982643990
Name: revenue, dtype: int64

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [35]:
answers['16'] = '3. Matt Damon'
high_budget = data[data.budget > data.budget.mean()]
high_budget_exploded = high_budget.explode('list_cast')
high_budget_exploded.list_cast.value_counts().head(1)

Matt Damon    18
Name: list_cast, dtype: int64

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [36]:
answers['17'] = '2. Action'
cage_genres = data[data.cast.str.contains('Nicolas Cage')][['list_genre']]
cage_genres_explode= cage_genres.explode('list_genre')
cage_genres_explode.list_genre.value_counts().head(1)

Action    17
Name: list_genre, dtype: int64

# 18. Самый убыточный фильм от Paramount Pictures

In [37]:
answers['18'] = '1. K-19: The Widowmaker (tt0267626)'
data[data.production_companies.str.contains('Paramount Pictures')][['imdb_id','original_title','profit']].sort_values('profit').head(1)

,imdb_id,original_title,profit
925,tt0267626,K-19: The Widowmaker,-64831034


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [38]:
answers['19'] = '5. 2015'
data.groupby(by=['release_year'])['imdb_id','original_title','profit'].sum().sort_values(by='profit',ascending=False).head(1)

,profit
release_year,
2015,18668572378


# 20. Какой самый прибыльный год для студии Warner Bros?

In [39]:
answers['20'] = '1. 2014'
data[data.production_companies.str.contains('Warner Bros')].groupby(by=['release_year'])['imdb_id','original_title','profit'].sum().sort_values(by='profit',ascending=False).head(1)

,profit
release_year,
2014,2295464519


# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [40]:
answers['21'] = '4. Сентябрь'

data.month.value_counts().head(1)

9    227
Name: month, dtype: int64

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [41]:
answers['22'] = '2. 450'
# Первый вариант
month_counts = data.month.value_counts()
summary_counts = month_counts[6] + month_counts[7] + month_counts[8]
summary_counts

450

In [42]:
# Второй вариант
data[data.month.isin([6,7,8])].imdb_id.count()

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [43]:
answers['23'] = '5. Peter Jackson'
data[data.month.isin([1,2,12])].list_director.value_counts().head(1)

[Peter Jackson]    7
Name: list_director, dtype: int64

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [44]:
answers['24'] = '5. Four By Two Productions'
data['title_len'] = data.original_title.map(lambda x: len(x))
data.groupby(by=['production_companies'])[['title_len']].mean().sort_values('title_len',ascending=False).head(1)

,title_len
production_companies,
Twentieth Century Fox Film Corporation|Four By Two Productions,83.0


# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [45]:
answers['25'] = '3. Midnight Picture Show'
data['overview_len'] = data.overview.map(lambda x: len(x))
data.groupby(by=['production_companies'])[['overview_len']].mean().sort_values('overview_len',ascending=False).head(1)

,overview_len
production_companies,
Lions Gate Films|Lakeshore Entertainment|GreeneStreet Films|Midnight Picture Show,1000.0


# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [46]:
answers['26'] = '1. Inside Out, The Dark Knight, 12 Years a Slave'
data[data.vote_average > data.quantile(0.99, numeric_only=True).vote_average].original_title


9                                          Inside Out
34                                               Room
118                                      Interstellar
119                           Guardians of the Galaxy
125                                The Imitation Game
128                                         Gone Girl
138                          The Grand Budapest Hotel
370                                         Inception
599                                   The Dark Knight
872                                       The Pianist
1081    The Lord of the Rings: The Return of the King
1183                          The Wolf of Wall Street
1191                                 12 Years a Slave
1800                                          Memento
Name: original_title, dtype: object

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [47]:
answers['27'] = '5. Daniel Radcliffe & Rupert Grint'
from itertools import combinations
actor_list = data.list_cast.tolist()
combo_list=[]
for i in actor_list:
    for j in combinations(i, 2):
        combo_list.append(' & '.join(j))
combo_list = pd.DataFrame(combo_list)
combo_list.columns = ['actor_combinations']
combo_list.actor_combinations.value_counts().head(10)

Daniel Radcliffe & Emma Watson        8
Daniel Radcliffe & Rupert Grint       8
Rupert Grint & Emma Watson            7
Johnny Depp & Helena Bonham Carter    6
Ben Stiller & Owen Wilson             6
Hugh Jackman & Ian McKellen           5
Kristen Stewart & Robert Pattinson    5
Kristen Stewart & Taylor Lautner      5
Robert Pattinson & Taylor Lautner     5
Adam Sandler & Kevin James            5
Name: actor_combinations, dtype: int64

ВАРИАНТ 2

# Submission

In [48]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': '2. Gods and Generals (tt0279111)',
 '3': '3. Winnie the Pooh  (tt1449283)',
 '4': '2. 110',
 '5': '1. 107',
 '6': '5. Avatar (tt0499549)',
 '7': '5. The Lone Ranger (tt1210819)',
 '8': '1. 1478',
 '9': '4. The Dark Knight (tt0468569)',
 '10': '5. The Lone Ranger (tt1210819)',
 '11': '3. Drama',
 '12': '1. Drama',
 '13': '5. Peter Jackson',
 '14': '3. Robert Rodriguez',
 '15': '3. Chris Hemsworth',
 '16': '3. Matt Damon',
 '17': '2. Action',
 '18': '1. K-19: The Widowmaker (tt0267626)',
 '19': '5. 2015',
 '20': '1. 2014',
 '21': '4. Сентябрь',
 '22': '2. 450',
 '23': '5. Peter Jackson',
 '24': '5. Four By Two Productions',
 '25': '3. Midnight Picture Show',
 '26': '1. Inside Out, The Dark Knight, 12 Years a Slave',
 '27': '5. Daniel Radcliffe & Rupert Grint'}

In [49]:
# и убедиться что ни чего не пропустил)
len(answers)

27